## Отзывы на фильм "Железный человек 3"
Для работы я выбрала рецензии на этот фильм, потому что, как известно, чем дальше фильм, тем ниже рейтинг. При этом ЖЧ-3 достаточно популярен, чтобы набрать побольше рецензий. Более того, в данном случае легко набрать рецензий для проверки - в качестве них я беру рецензии на ЖЧ-2.

In [1]:
from bs4 import BeautifulSoup 
import requests, re, nltk
from pymorphy2 import MorphAnalyzer
morph = MorphAnalyzer()
import collections
from sklearn.metrics import accuracy_score

Собираем ссылки на все рецензии (положительные и отрицательные, из разных ссылок их проще выкачивать). При этом дальше мы функциями обкачиваем ссылки и чистим их от лишних вещей.

###### Важно, что плохие отзывы мы ещё чистим от отзывов, оценка за которые 6 или выше, потому что это может оказаться слишком "нейтральными". Дисбаланс между количеством положительных и отрицательных отзывов компенсируем за счёт ограничения количества отзывов, которое мы берём на уровне фильтрации

In [2]:
bad_revs_link = 'https://www.kinopoisk.ru/film/462762/reviews/ord/rating/status/bad/perpage/100/'
good_revs_link = 'https://www.kinopoisk.ru/film/462762/reviews/ord/rating/status/good/perpage/100/'
good_check_link = 'https://www.kinopoisk.ru/film/411924/reviews/ord/rating/status/good/perpage/50/'
bad_check_link = 'https://www.kinopoisk.ru/film/411924/reviews/ord/rating/status/bad/perpage/50/'

In [3]:
def get_reviews(link):
    response = requests.get(link)
    soup = BeautifulSoup(response.text, 'html.parser')
    html = soup.find_all('span', {'itemprop': 'reviewBody'})
    return html

In [4]:
def filter_bad(reviews):
    too_good = ['6 из 10', '7 из 10', '6.5 из 10', '7.5 из 10']
    bad_revs = []
    for rev in reviews:
        for mark in too_good:
            if mark in str(rev):
                reviews.remove(rev)
    return reviews

In [5]:
def filter_revs(reviews, max_len):
    filtered = []
    reg = re.compile('<.*?>')
    counter = 0
    for rev in reviews:
        if counter < max_len:
            clean_rev = re.sub(reg, '', str(rev))
            filtered.append(clean_rev)
            counter = counter + 1
    return filtered

In [6]:
def getting_set(revs):
    words = []
    for rev in revs:
        for word in nltk.word_tokenize(rev.lower()): 
            if word.isalpha():
                lemma = morph.parse(word)[0].normal_form
                words.append(lemma)
    count = collections.Counter(words).most_common(1400)
    return set(list(dict(count)))

После фильтрации и доставания множеств слов пишем функцию, которая будем с этими множествами опреедлять тональность.

In [7]:
def check_tone(review):
    good_count = 0
    bad_count = 0
    result = str()
    for word in nltk.word_tokenize(review):
        if word.isalpha():
            lemma = morph.parse(word)[0].normal_form
            if lemma in full_bad_words:
                bad_count += 1
            elif lemma in full_good_words:
                good_count += 1
    if good_count > bad_count:
        result = 'good'
    elif bad_count >= good_count:
        result = 'bad'
    return result

In [8]:
bad_reviews_raw = filter_bad(get_reviews(bad_revs_link))
good_reviews_raw = get_reviews(good_revs_link)
bad_check = filter_bad(get_reviews(bad_check_link))
good_check = get_reviews(good_check_link)

In [9]:
bad_revs = filter_revs(bad_reviews_raw, len(bad_reviews_raw))
good_revs = filter_revs(good_reviews_raw, len(bad_reviews_raw))
good_check_revs = filter_revs(good_check, 25)
bad_check_revs = filter_revs(bad_check, 25)
full_bad_words = getting_set(bad_revs)
full_good_words = getting_set(good_revs)

In [10]:
for_check = {'bad': bad_check_revs, 'good': good_check_revs}

In [11]:
double = []
for word in full_bad_words:
    if word in full_good_words:
        double.append(word)

In [12]:
for d in double:
    if d in full_bad_words:
        full_bad_words.remove(d)
    if d in full_good_words:
        full_good_words.remove(d)  

В результате код выдаёт точность программы. программу можно улучшить по-разному: например, можно было бы отсеивать из отрицательных отзывов еще и отзывы с 4-5 из 10, потому что такие отзывы бывают нейтральными + можно было бы отсеивать отзывы на 7-8 из 10 среди хороших. Было бы хорошо добавить больше данных: например, обкачать все фильмы марвел таким образом (если мыслить о близких отзывах). 

In [13]:
tones = ['bad', 'good']
results = []
gold = []
for tone in tones:
    for rev in for_check[tone]:
        results.append(check_tone(rev))
        gold.append(tone)
print('Точность: ', accuracy_score(results, gold))

Точность:  0.76
